In [2]:
import os
from time import time 
import random
#import sys
#import operator
import numpy as np
import pandas as pd
from scipy import sparse
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
#from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss, accuracy_score
#from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [3]:
import matplotlib.pylab as plt
%matplotlib inline

In [4]:
def modelfit(alg, train_X, train_y, useTrainCV=True, cv_folds=5, early_stopping_rounds=30):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(train_X,label=train_y)
        #xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        start_train = time()
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='logloss', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    index = range(train_X.shape[0])
    random.shuffle(index)
    train_len = int(0.8*train_X.shape[0])
    train_index = index[:train_len]
    val_index = index[train_len:]
    #Fit the algorithm on the data
    alg.fit(train_X[train_index,:], train_y[train_index,],eval_metric='logloss')
    end_train = time()
    print('training time: %.4g' % (end_train - start_train))
        
    #Predict training set:
    val_predictions = alg.predict(train_X[val_index,:])
    val_predprob = alg.predict_proba(train_X[val_index,:])[:,1]
        
    #Print model report:
    print "\nModel Report"
    print "Accuracy(val) : %.4g" % accuracy_score(train_y[val_index,], val_predictions)
    print "logloss Score (val): %f" % log_loss(train_y[val_index,], val_predprob)
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [24]:
preds = xgb1.predict(test_X)

In [27]:
accuracy_score(test_y,preds)

0.76000000000000001

In [22]:
cvresult

,test-logloss-mean,test-logloss-std,train-logloss-mean,train-logloss-std
0,0.680836,0.003198,0.656950,0.002566
1,0.670279,0.005958,0.623260,0.004267
2,0.658826,0.006989,0.593185,0.006427
3,0.646850,0.007072,0.562975,0.007304
4,0.640163,0.009004,0.537867,0.007488
5,0.632632,0.008216,0.513151,0.008021
6,0.626698,0.010264,0.492119,0.007942
7,0.619197,0.010340,0.471745,0.007512
8,0.613525,0.011675,0.452709,0.007847
9,0.609673,0.013546,0.434061,0.007077


In [7]:
xgb1 = XGBClassifier(
 learning_rate =0.03,
 n_estimators=400,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.7,
 colsample_bytree=0.7,
 objective= 'binary:logistic',
 #nthread=4,
 #scale_pos_weight=1,
 seed=59)

training time: 82.85

Model Report
Accuracy(val) : 0.7438
logloss Score (val): 0.526039


In [8]:
modelfit(xgb1, train_X, train_y)

list

In [21]:
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0, num_rounds=1000):
    param = {}
    param['objective'] = 'binary:logistic'
    param['eta'] = 0.1
    param['max_depth'] = 6
    param['silent'] = 1
    param['num_class'] = 2
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plist = list(param.items())
    xgtrain = xgb.DMatrix(train_X,label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test')]
        train_start = time()
        cvresult = xgb.cv(plist, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='logloss', early_stopping_rounds=early_stopping_rounds, show_progress=False)
        #model = xgb.train(plist,xgtrain,num_rounds,watchlist,early_stopping_rounds=20)
        train_end = time()
    else:
        xgtest = xgb.DMatrix(test_X)
        train_start = time()
        model = xgb.train(plist, xgtrain, num_rounds)
        train_end = time()
    
    test_start = time()
    pred_test_y = model.predict(xgtest)
    test_end = time()
    return pred_test_y, model, (train_end - train_start), (test_end - test_start)

In [5]:
input_path = '../../localData/prj3/training_data/sift_features/sift_features_1000.csv'
fix_test_path = '../output/prediction_inceptionV3.csv'
test_images = pd.read_csv(fix_test_path)['image']
test_images = [x.split('.')[0] for x in test_images.tolist()]

total_df = pd.read_csv(input_path).transpose()

labels = [1 for i in range(1000)] + [0 for i in range(1000)]
total_df['label'] = labels

In [6]:
# for ensemble purpose make sure test set the same as in inceptionV3 model
test_df = total_df.ix[total_df.index.isin(test_images)]
train_df = total_df.ix[~total_df.index.isin(test_images)]

train_X = train_df.ix[:,:1000]
train_y = train_df['label']

test_X = test_df.ix[:,:1000]
test_y = test_df['label']

train_X = sparse.csr_matrix(train_X.values)
test_X = sparse.csr_matrix(test_X.values)

train_y = np.array(train_y)
test_y = np.array(test_y)

In [22]:
preds, _ , training_time, predicting_time = runXGB(train_X, train_y, test_X, num_rounds=400)

In [23]:
print('training time is:'+str(round(training_time,2))+'seconds;\npredicting time is:'+
str(round(predicting_time,2))+'seconds.')

training time is:50.34seconds;
predicting time is:0.01seconds.


In [24]:
out_df = pd.DataFrame(preds)

In [25]:
out_df.head()

,0,1
0,0.930725,0.069275
1,0.757777,0.242223
2,0.432586,0.567414
3,0.802996,0.197004
4,0.065175,0.934825


In [14]:
train_X.shape

(1600, 5000)

In [15]:
train_y.shape

(1600,)

In [17]:
test_X.shape

(400, 5000)

In [18]:
train_y

array([1, 1, 1, ..., 0, 0, 0])

In [20]:
param = {}
param['objective'] = 'multi:softprob'
param['eta'] = 0.1
param['max_depth'] = 6
param['silent'] = 1
param['num_class'] = 3
param['eval_metric'] = "mlogloss"
param['min_child_weight'] = 1
param['subsample'] = 0.7
param['colsample_bytree'] = 0.7
param['seed'] = 0
num_rounds = 400